Install Libraries

In [ ]:
!pip install pybaseball pandas numpy xgboost scikit-learn


Load Libraries

In [ ]:
from pybaseball import batting_stats
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

Step 1: Pull 2010-2023 Batting Stats

In [ ]:
# Get batting stats for multiple years (2015-2023)
years = list(range(2015, 2024))
batting_data = pd.concat([batting_stats(y) for y in years], ignore_index=True)

# Select key batting metrics (adding new high-impact features)
batting_features = [
    "IDfg", "Season", "wOBA", "xwOBA", "ISO+", "EV", "HardHit%", "WPA", "REW", "OBP", "SLG", "ISO", "Age",
    "WPA/LI", "Off", "xSLG", "wRAA", "RE24", "maxEV",  # Existing features
    "BB%", "K%", "Spd"  # New features for feature engineering
]

# Keep only relevant columns
batting_data = batting_data[batting_features]

# Rename columns to match expected format
batting_data.rename(columns={
    "IDfg": "player_id",
    "Season": "year",
    "HardHit%": "hard_hit_rate",
    "maxEV": "max_exit_velocity",
    "BB%": "bb_rate",
    "K%": "k_rate",
    "Spd": "speed_score"
}, inplace=True)

# Ensure correct data types
batting_data["year"] = batting_data["year"].astype(int)

print(f"✅ Pulled batting stats from {years[0]} to {years[-1]} (Statcast Era)")
print(f"✅ Dataset Shape After Filtering: {batting_data.shape}")
print(batting_data.head())


Step 2: Shift wOBA to Predict Next Season

In [ ]:
# Shift wOBA forward to predict next season's performance
batting_data["next_year_woba"] = batting_data.groupby("player_id")["wOBA"].shift(-1)

# Drop rows where next year's wOBA is NaN (i.e., last recorded season for a player)
batting_data = batting_data.dropna(subset=["next_year_woba"])

# Ensure year column is integer type
batting_data["year"] = batting_data["year"].astype(int)

print("✅ Successfully Shifted wOBA Forward to Predict Next Season")
print(f"✅ Dataset Shape After Shift: {batting_data.shape}")
print(batting_data.head())


Create new features

In [ ]:
# Feature Engineering: Compute new advanced metrics

# BB/K Ratio (Plate Discipline)
if "bb_rate" in batting_data.columns and "k_rate" in batting_data.columns:
    batting_data["BB_K_Ratio"] = batting_data["bb_rate"] / batting_data["k_rate"]
    print("✅ Added BB/K Ratio")

# Speed Influence (Sprint Speed Factor)
if "speed_score" in batting_data.columns:
    batting_data["Speed_Impact"] = batting_data["speed_score"] * batting_data["ISO+"]
    print("✅ Added Speed-Adjusted ISO")

# Fill missing values
batting_data.fillna(0, inplace=True)

print(f"✅ New Features Engineered! Updated Shape: {batting_data.shape}")

# Sort data to ensure correct shifting order
batting_data = batting_data.sort_values(by=["player_id", "year"])

# Compute year-over-year change in wOBA
batting_data["wOBA_change"] = batting_data.groupby("player_id")["wOBA"].diff()

# Define the weights for rolling wOBA
weights_3Y = [0.5, 0.3, 0.2]  # More recent years have higher weight
weights_5Y = [0.4, 0.25, 0.15, 0.1, 0.1]

def compute_weighted_rolling_wOBA(df, player_col="player_id", year_col="year", woba_col="wOBA"):
    """
    Computes weighted rolling averages for wOBA over 3-year and 5-year windows.
    """
    df = df.sort_values(by=[player_col, year_col])  # Ensure sorting for rolling calculations

    # Initialize rolling averages
    df["weighted_wOBA_3Y"] = np.nan
    df["weighted_wOBA_5Y"] = np.nan

    # Process each player separately
    for player in df[player_col].unique():
        player_mask = df[player_col] == player
        player_data = df.loc[player_mask, woba_col]

        # Compute 3-Year Weighted Rolling Average
        if len(player_data) >= 3:
            df.loc[player_mask, "weighted_wOBA_3Y"] = (
                player_data.rolling(3, min_periods=3)
                .apply(lambda x: np.dot(x, weights_3Y[-len(x):]), raw=True)
            )

        # Compute 5-Year Weighted Rolling Average
        if len(player_data) >= 5:
            df.loc[player_mask, "weighted_wOBA_5Y"] = (
                player_data.rolling(5, min_periods=5)
                .apply(lambda x: np.dot(x, weights_5Y[-len(x):]), raw=True)
            )

    return df

# Apply function to dataset
batting_data = compute_weighted_rolling_wOBA(batting_data)

# Fill any remaining NaNs with 0
batting_data.fillna(0, inplace=True)

# Step 2: Implement Age-Based Decline Factor

def age_decline_factor(age):
    """
    Models decline after age 30 using a logistic decay function.
    Players peak at 27-29, decline past 30.
    """
    return 1 / (1 + np.exp((age - 30) / 3))  # Smooth decline after 30

batting_data['age_decline_factor'] = batting_data['Age'].apply(age_decline_factor)

# Step 3: Update Feature List
batting_features += ['age_decline_factor']

batting_data.head()

Create League avg features

In [ ]:
# Define the stats we want to compare to league averages
stats_to_compare = ["wOBA", "ISO", "EV", "OBP", "SLG", "hard_hit_rate", "max_exit_velocity"]

# Compute league averages per year
league_averages = batting_data.groupby("year")[stats_to_compare].mean().reset_index()
league_averages.rename(columns={col: f"league_avg_{col}" for col in stats_to_compare}, inplace=True)

# Merge league averages back into the player data
batting_data = batting_data.merge(league_averages, on="year", how="left")

for stat in stats_to_compare:
    batting_data[f"rel_{stat}"] = batting_data[stat] - batting_data[f"league_avg_{stat}"]



Step 3: Create Training Set

In [ ]:
from sklearn.model_selection import train_test_split

# Updated feature selection (adding newly engineered features)


features = [
    "WPA/LI", "REW", "RE24", "xSLG", "Off", "EV", "wRAA", "ISO+",
    "max_exit_velocity", "BB_K_Ratio", "WPA", "Speed_Impact",
    "rel_wOBA", "rel_ISO", "rel_EV", "rel_OBP", "rel_SLG", "rel_hard_hit_rate",
    "Age", "wOBA_change", "weighted_wOBA_3Y", "age_decline_factor", "weighted_wOBA_5Y"
]


# Define X and y for model training
X = batting_data[features]
y = batting_data["next_year_woba"]

# Fill missing values
X = X.fillna(0)
y = y.fillna(0)

print(f"✅ Features used: {features}")
print(f"✅ Training Data Shape: {X.shape}")

# Train-test split (Re-run this before training)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✅ Train size: {X_train.shape}, Test size: {X_test.shape}")
print(f"✅ Features used: {features}")


Train XGBoost Model

In [ ]:


# Define hyperparameter grid
param_grid = {
    "n_estimators": [400, 500, 600],  # Increase estimators slightly
    "max_depth": [3, 4, 5],  # Increase depth
    "learning_rate": [0.005, 0.01, 0.02],  # Try slightly higher values
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8]
}




# Initialize XGBoost model
xgb = XGBRegressor(objective="reg:squarederror")

# Grid Search with Cross-Validation
grid_search = GridSearchCV(xgb, param_grid, scoring="r2", cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best model
best_xgb = grid_search.best_estimator_

# Print best parameters
print(f"🚀 Best Parameters: {grid_search.best_params_}")

# Train best model on full dataset
best_xgb.fit(X_train, y_train)

# Evaluate performance
train_score = best_xgb.score(X_train, y_train)
test_score = best_xgb.score(X_test, y_test)

print(f"🎯 Optimized Training Score: {train_score:.3f}")
print(f"📉 Optimized Test Score: {test_score:.3f}")


Predict Adley's 2024 wOBA using 2023 statistics

In [ ]:
# Adley Rutschman's FanGraphs ID
adley_id = 17621  # Replace with actual ID if different

# Filter Adley's 2023 stats
adley_2023 = batting_data[(batting_data["player_id"] == adley_id) & (batting_data["year"] == 2023)]

# Compute average feature values
adley_avg = adley_2023[features].mean().values.reshape(1, -1)

# Predict 2024 wOBA
predicted_wOBA = best_xgb.predict(adley_avg)[0]

print(f"🎯 Projected wOBA for Adley Rutschman (2024): {predicted_wOBA:.3f}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Get feature importance from trained XGBoost model
importance_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": best_xgb.feature_importances_
}).sort_values(by="Importance", ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x=importance_df["Importance"], y=importance_df["Feature"])
plt.xlabel("Feature Importance Score")
plt.ylabel("Feature")
plt.title("Feature Importance in XGBoost Model")
plt.show()


In [ ]:
import numpy as np

# Get predictions
y_pred = best_xgb.predict(X_test)

# Compute residuals
residuals = y_test - y_pred

# Plot residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, bins=30, kde=True)
plt.xlabel("Residual (Actual wOBA - Predicted wOBA)")
plt.ylabel("Frequency")
plt.title("Residual Distribution")
plt.show()


In [ ]:
# Identify high-error predictions
batting_data["residuals"] = y_test - y_pred

# Sort by absolute residual error
high_residuals = batting_data.loc[X_test.index].copy()
high_residuals["abs_residuals"] = high_residuals["residuals"].abs()
high_residuals = high_residuals.sort_values(by="abs_residuals", ascending=False)

print("🔍 Players with Largest Prediction Errors:")
print(high_residuals[["player_id", "next_year_woba", "residuals"]].head(10))
